# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os

# Set your preferred name
USER = "Jinglin"
# Set the data and output directories
DATA_ROOT = '../../DATA'
OUTPUT_ROOT = '../../output1'
TRAIT = 'Stomach Cancer'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [3]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

['TCGA_Pancreatic_Cancer_(PAAD)',
 'TCGA_Bladder_Cancer_(BLCA)',
 '.DS_Store',
 'TCGA_Testicular_Cancer_(TGCT)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Breast_Cancer_(BRCA)',
 'TCGA_Stomach_Cancer_(STAD)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Glioblastoma_(GBM)']

If no match is found, jump directly to GEO in Part 2.2

In [5]:
trait_subdir = "TCGA_Stomach_Cancer_(STAD)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.STAD.sampleMap_STAD_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.STAD.sampleMap_HiSeqV2_PANCAN.gz')

In [6]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [7]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [8]:
clinical_data_cols[:10]

['CDE_ID_3226963',
 '_INTEGRATION',
 '_PATIENT',
 '_cohort',
 '_primary_disease',
 '_primary_site',
 'additional_pharmaceutical_therapy',
 'additional_radiation_therapy',
 'additional_surgery_locoregional_procedure',
 'additional_surgery_metastatic_procedure']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [9]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['CDE_ID_3226963', '_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'additional_surgery_metastatic_procedure', 'age_at_initial_pathologic_diagnosis', 'anatomic_neoplasm_subdivision', 'antireflux_treatment', 'antireflux_treatment_type', 'barretts_esophagus', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'city_of_p

In [10]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis',
                      'days_to_birth']
candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [11]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [70.0, 51.0, 51.0, 62.0, 52.0],
 'days_to_birth': [nan, nan, -18698.0, -22792.0, -19014.0]}

In [12]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [13]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}

In [14]:
gender_col = 'gender'

In [15]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [16]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

15 input query terms found dup hits:	[('GTF2IP1', 2), ('RBMY1A3P', 3), ('HERC2P2', 3), ('WASH3P', 3), ('NUDT9P1', 2), ('SNORD104', 2), ('
154 input query terms found no hit:	['C16orf13', 'C16orf11', 'LOC100272146', 'LOC339240', 'NACAP1', 'LOC441204', 'KLRA1', 'FAM183A', 'FA
9 input query terms found dup hits:	[('SUGT1P1', 2), ('SNORD127', 2), ('SNORA62', 4), ('IFITM4P', 8), ('HLA-DRB6', 2), ('FUNDC2P2', 2), 
190 input query terms found no hit:	['NARFL', 'NFKBIL2', 'LOC150197', 'TMEM84', 'LOC162632', 'PPPDE1', 'PPPDE2', 'C1orf38', 'C1orf31', '
20 input query terms found dup hits:	[('PIP5K1P1', 2), ('SNORA40', 2), ('SNORD116-24', 2), ('SNORD116-25', 2), ('SNORD116-23', 2), ('SNOR
149 input query terms found no hit:	['FAM153C', 'C9orf167', 'CLK2P', 'CCDC76', 'CCDC75', 'CCDC72', 'HIST3H2BB', 'PRAC', 'LOC285780', 'LO
18 input query terms found dup hits:	[('SNORD58C', 2), ('UOX', 2), ('UBE2Q2P1', 3), ('PPP4R1L', 2), ('SNORD63', 3), ('SNORD116-22', 2), (
158 input query terms found no hit:	['

In [17]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Stomach Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-3M-AB46-01,1,70.0,1.0,2.168708,-0.956226,-0.531035,0.135928,0.349322,0.14139,-0.419494,...,-1.531086,-0.086682,0.213122,-1.587117,-0.417833,1.55901,-1.051477,-0.131827,0.312605,-2.890033
TCGA-3M-AB47-01,1,51.0,1.0,1.309708,1.160674,-0.037135,0.210328,0.313322,-0.28301,-0.415794,...,-1.592886,-0.086682,-0.254978,0.973083,0.544567,-0.42039,-0.167577,1.148673,0.948205,4.198067
TCGA-B7-5816-01,1,51.0,0.0,-0.045892,-2.968626,-0.531035,-0.206872,-0.105678,0.15469,-0.937094,...,0.165814,-0.086682,-0.748878,-1.587117,0.180067,-1.28139,-0.364377,0.564673,0.272405,1.328567
TCGA-B7-5818-01,1,62.0,1.0,0.076608,5.843574,-0.531035,-0.190272,-0.647478,-0.02321,-0.576694,...,-1.364086,-0.086682,0.818922,-1.587117,1.134367,-1.28139,-0.446777,-0.050027,1.929905,-2.179533
TCGA-B7-A5TI-01,1,52.0,1.0,0.046908,5.233774,-0.531035,-0.108772,-0.233578,0.21699,-1.040094,...,-2.090786,-0.086682,-0.748878,-1.587117,0.381167,2.98761,0.239123,1.278873,0.907905,1.724767


In [18]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 445 samples.


In [19]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Stomach Cancer', the least common label is '0' with 35 occurrences. This represents 7.87% of the dataset.
The distribution of the feature 'Stomach Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 58.0
  50% (Median): 67.0
  75%: 73.0
Min: 30.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 157 occurrences. This represents 35.28% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [20]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [21]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [22]:
dataset = 'GEO'
trait_subdir = "Stomach-Cancer"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE207846', 'GSE172159', 'GSE147163', 'GSE211704', 'GSE172197']

Repeat the below steps for all the accession numbers

In [6]:
cohort = accession_num = "GSE172197"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('../../DATA/GEO/Stomach-Cancer/GSE172197/GSE172197_family.soft.gz',
 '../../DATA/GEO/Stomach-Cancer/GSE172197/GSE172197_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [9]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"mRNA expression profiles of newly established 49 gastric cancer cell lines."
!Series_summary	"Establishment and molecular characterization of 49 peritoneally-metastatic gastric cancer cell lines from 18 patients’ ascites."
!Series_summary	"We performed comprehensive transcriptome analyses using microarrays of our established gastric cancer cell lines."
!Series_overall_design	"49 cancer cell lines"


In [10]:
clinical_data.head()

,!Sample_geo_accession,GSM5243830,GSM5243831,GSM5243832,GSM5243833,GSM5243834,GSM5243835,GSM5243836,GSM5243844,GSM5243845,...,GSM5243876,GSM5243877,GSM5243878,GSM5243879,GSM5243880,GSM5243881,GSM5243882,GSM5243883,GSM5243884,GSM5243885
0,!Sample_characteristics_ch1,cell line: NSC-10C,cell line: NSC-10X1A,cell line: NSC-10X1aA,cell line: NSC-10X1aF,cell line: NSC-10X1aX1,cell line: NSC-10X1aX1a,cell line: NSC-10X1F,cell line: NSC-11C,cell line: NSC-11X1,...,cell line: NSC-40CX1,cell line: NSC-40CX1a,cell line: NSC-44CA,cell line: NSC-44CF,cell line: NSC-44CX1,cell line: NSC-44CX1a,cell line: NSC-47C,cell line: NSC-47CX1F,cell line: NSC-56C-1,cell line: NSC-56C-2


In [11]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['cell line: NSC-10C',
  'cell line: NSC-10X1A',
  'cell line: NSC-10X1aA',
  'cell line: NSC-10X1aF',
  'cell line: NSC-10X1aX1',
  'cell line: NSC-10X1aX1a',
  'cell line: NSC-10X1F',
  'cell line: NSC-11C',
  'cell line: NSC-11X1',
  'cell line: NSC-11X1a',
  'cell line: NSC-15CA',
  'cell line: NSC-15CF',
  'cell line: NSC-16C',
  'cell line: NSC-16CX1F',
  'cell line: NSC-17CA',
  'cell line: NSC-17CF',
  'cell line: NSC-18C-1',
  'cell line: NSC-18C-2',
  'cell line: NSC-18C-3',
  'cell line: NSC-20C',
  'cell line: NSC-20CX1',
  'cell line: NSC-20CX1a',
  'cell line: NSC-20CX2',
  'cell line: NSC-20CX2a',
  'cell line: NSC-24C',
  'cell line: NSC-24CX1a',
  'cell line: NSC-26C-1',
  'cell line: NSC-26C-2',
  'cell line: NSC-28C',
  'cell line: NSC-28CX1',
  'cell line: NSC-28CX1a',
  'cell line: NSC-32C',
  'cell line: NSC-32CX1',
  'cell line: NSC-32CX1a',
  'cell line: NSC-35C-1',
  'cell line: NSC-35C-2',
  'cell line: NSC-38C-1',
  'cell line: NSC-38C-2',
  'cell line: N

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [42]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.
   NOTE: Function "convert_trait" should return 1 or 0.
   
Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Stomach Cancer\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Stomach Cancer\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [12]:
# is_gene_availabe = True
is_gene_availabe = False
trait_row_id = 0
trait_type = 'binary'

age_row_id = None
# age_row_id = 1

gender_row_id = None
# gender_row_id = 2

In [13]:
is_available = is_gene_availabe and (trait_row_id is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

This cohort is not usable. Please skip the following steps and jump to the next accession number.


In [47]:
def convert_trait(value):
    if value == 'disease state: Gastric cancer':
          return 1
    else:
          return 0
    
def convert_age(value):
    try:
        # Split the string and extract the age part
        age_str = value.split(': ')[-1]
        return int(age_str)  # Convert the extracted age to an integer
    except (ValueError, TypeError, IndexError):
        return None  # Unable to convert or unknown age
    
def convert_gender(value):
    if isinstance(value, str):
        if 'F' in value:
            return 0  # Female
        elif 'M' in value:
            return 1  # Male
    return None  # Unknown or unhandled cases


In [48]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row_id, age_row_id, gender_row_id,
                                                      convert_trait, convert_age, convert_gender)
selected_clinical_data.head()

,GSM6499512,GSM6499513,GSM6499514,GSM6499515,GSM6499516,GSM6499517,GSM6499518,GSM6499519,GSM6499520,GSM6499521,...,GSM6552206,GSM6552207,GSM6552208,GSM6552209,GSM6552210,GSM6552211,GSM6552212,GSM6552213,GSM6552214,GSM6552215
Stomach Cancer,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


### Genetic data preprocessing and final filtering

In [52]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,ID,GSM6499512,GSM6499513,GSM6499514,GSM6499515,GSM6499516,GSM6499517,GSM6499518,GSM6499519,GSM6499520,...,GSM6552206,GSM6552207,GSM6552208,GSM6552209,GSM6552210,GSM6552211,GSM6552212,GSM6552213,GSM6552214,GSM6552215
0,cg00000029,0.332572,0.511428,0.418744,0.293688,0.246596,0.303733,0.337906,0.263289,0.234159,...,0.391020,0.273057,0.250134,0.186785,0.534048,0.291826,0.393869,0.171678,0.401537,0.309824
1,cg00000108,0.903620,0.942419,0.907896,0.940411,0.945472,0.922329,0.939412,0.801059,0.926147,...,0.929448,0.899164,0.850762,0.970414,0.872145,0.907492,0.949298,0.933325,0.918855,0.930271
2,cg00000109,0.842498,0.782451,0.863456,0.829853,0.803795,0.841160,0.872059,0.823878,0.897155,...,0.845600,0.836350,0.867638,0.874303,0.908187,0.860378,0.883108,0.720102,0.802628,0.805208
3,cg00000165,0.493801,0.208688,0.297658,0.380990,0.662907,0.413185,0.477908,0.624963,0.604757,...,0.629443,0.189181,0.361495,0.635423,0.792375,0.329569,0.537443,0.550747,0.396046,0.456668
4,cg00000236,0.834600,0.894552,0.805181,0.862047,0.830529,0.807524,0.852917,0.833126,0.869469,...,0.878394,0.783567,0.854523,0.914413,0.845754,0.859671,0.858270,0.883156,0.804368,0.800849


In [53]:
gene_row_ids = genetic_data['ID'][:20].tolist()
gene_row_ids

['cg00000029',
 'cg00000108',
 'cg00000109',
 'cg00000165',
 'cg00000236',
 'cg00000289',
 'cg00000292',
 'cg00000321',
 'cg00000363',
 'cg00000622',
 'cg00000658',
 'cg00000714',
 'cg00000721',
 'cg00000734',
 'cg00000769',
 'cg00000807',
 'cg00000884',
 'cg00000905',
 'cg00000924',
 'cg00000948']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [54]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['cg00000029', 'cg00000108', 'cg00000109', 'cg00000165', 'cg00000236', 'cg00000289', 'cg00000292', 'cg00000321', 'cg00000363', 'cg00000622', 'cg00000658', 'cg00000714', 'cg00000721', 'cg00000734', 'cg00000769', 'cg00000807', 'cg00000884', 'cg00000905', 'cg00000924', 'cg00000948']\n"

If not required, jump directly to the gene normalization step

In [55]:
requires_gene_mapping = True

In [ ]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)[0]
    gene_annotation_summary = preview_df(gene_annotation)
gene_annotation_summary

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [ ]:
# if requires_gene_mapping:
f'''
As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
probe_id_key = key_name1
gene_symb_key = key_name2

Gene annotation dictionary:
{gene_annotation_summary}
'''

In [31]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'Symbol'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [32]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [33]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [34]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 50 samples.


In [35]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Stomach Cancer', the least common label is '1.0' with 50 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Stomach Cancer' in this dataset is severely biased.



True

In [36]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

In [37]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [4]:
# Check the information of usable cohorts
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH)
ranked_df

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,Xena,True,True,False,True,True,445.0,


In [5]:
# If both age and gender have available cohorts, select 'age' as the condition.
condition = 'Age'
filter_column = 'has_' + condition.lower()

condition_best_cohort, condition_ranked_df = filter_and_rank_cohorts(JSON_PATH, filter_column)
condition_best_cohort

'Xena'

In [6]:
condition_ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,Xena,True,True,False,True,True,445.0,


In [7]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, condition_best_cohort + '.csv'))
merged_data.head()

,Stomach Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
0,1,70.0,1.0,2.168708,-0.956226,-0.531035,0.135928,0.349322,0.14139,-0.419494,...,-1.531086,-0.086682,0.213122,-1.587117,-0.417833,1.55901,-1.051477,-0.131827,0.312605,-2.890033
1,1,51.0,1.0,1.309708,1.160674,-0.037135,0.210328,0.313322,-0.28301,-0.415794,...,-1.592886,-0.086682,-0.254978,0.973083,0.544567,-0.42039,-0.167577,1.148673,0.948205,4.198067
2,1,51.0,0.0,-0.045892,-2.968626,-0.531035,-0.206872,-0.105678,0.15469,-0.937094,...,0.165814,-0.086682,-0.748878,-1.587117,0.180067,-1.28139,-0.364377,0.564673,0.272405,1.328567
3,1,62.0,1.0,0.076608,5.843574,-0.531035,-0.190272,-0.647478,-0.02321,-0.576694,...,-1.364086,-0.086682,0.818922,-1.587117,1.134367,-1.28139,-0.446777,-0.050027,1.929905,-2.179533
4,1,52.0,1.0,0.046908,5.233774,-0.531035,-0.108772,-0.233578,0.21699,-1.040094,...,-2.090786,-0.086682,-0.748878,-1.587117,0.381167,2.98761,0.239123,1.278873,0.907905,1.724767


In [8]:
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [9]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [10]:
# Select appropriate models based on whether the dataset has batch effect.
# We experiment on two models for each branch. We will decide which one to choose later.

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [11]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 46.07% ± 8.32%


In [12]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


The cross-validation accuracy is 82.70% ± 3.38%


In [13]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


### 5. Discussion and report

In [14]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0095
Found 45 genes with non-zero coefficients associated with the trait 'Stomach Cancer' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
 Variable  Coefficient  Absolute Coefficient
   ARL17B    -1.301043              1.301043
     GNMT     1.048123              1.048123
    SCN2A     0.944232              0.944232
    CD177     0.939321              0.939321
 DEFB108B     0.878645              0.878645
    CDH26     0.872231              0.872231
    SSTR4    -0.815231              0.815231
    ASB18     0.696554              0.696554
    SPEM1    -0.630551              0.630551
   ZNF492     0.579907              0.579907
    WNT16    -0.543612              0.543612
    SATL1     0.533077              0.533077
    SIRPD    -0.523171              0.523171
    UBAC1     0.471536              0.471536
    INSL6     0.447704              0.447704
   GAG

/Users/ollie/Documents/JupyterProjects/AI4Science_gold_standard/stage1/Jinglin/utils.py:469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [15]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0095
Found 52 significant genes associated with the trait 'Stomach Cancer' conditional on the factor 'Age', with corrected p-value < 0.05:
 Variable  Coefficient      p_value  corrected_p_value
    TREM2     0.108970 4.751191e-14       8.238090e-10
      BGN     0.126377 5.442266e-11       4.718172e-07
  COL10A1     0.099321 8.491581e-11       4.907851e-07
   HPCAL4    -0.060357 4.208342e-09       1.824211e-05
    INHBA     0.092665 6.916513e-09       2.398508e-05
 ADAMTS18     0.058923 9.512741e-09       2.749023e-05
    APOC1     0.074443 1.638887e-07       4.059524e-04
   PKMYT1     0.102345 4.276538e-07       8.238989e-04
      SDS     0.065192 4.082816e-07       8.238989e-04
   OLFML1    -0.096786 5.857173e-07       1.015575e-03
   HOXC11     0.047712 1.163186e-06       1.833499e-03
  ONECUT2     0.052472 1.395640e-06       2.016583e-03
     WIF1    -0.044932 1.608712e-06       2.145650e-03
    CLDN7    

/Users/ollie/Documents/JupyterProjects/AI4Science_gold_standard/stage1/Jinglin/utils.py:478: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
